# Тестирование API

In [ ]:
%load_ext autoreload
%autoreload 2

import os

In [ ]:
path_list = os.getcwd().split('\\')
if path_list[-2] == 'recs' and path_list[-1] == 'recs':
    pass
else:
    os.chdir('..')
    print(os.getcwd())

In [ ]:
SEED = 1

## Импорт модулей

In [ ]:
import dataset
import preprocessing
import models
import similarity_search
import augmentation

#### Настройки для pipeline:

In [ ]:
dataset = dataset.load_video_games()

preprocessing_list = [
    preprocessing.BaseCleaner(remove_number=False),
    preprocessing.BaseNormalizer(
        'english',
        remove_stopwords=True,
        number_extract=True,
        lemmatize=True,
    ),
]

model = models.FastTextWrapperModel(
    min_count=1,
    vector_size=200,
    window=2,
    sg=1,
    hs=1,
    epochs=70,
    min_n=0,
    seed=SEED,
)

searcher1 = similarity_search.FaissSearch
searcher2 = similarity_search.NearestNeighborsSearch

#### Настройки для валидации:

In [ ]:
augmentation_transforms = [
    augmentation.MisspellingAugmentation(
        add_syms={'p': 0.01, 'language': 'english'},
        change_syms={'p': 0.01, 'language': 'english'},
        delete_syms={'p': 0.01},
        multiply_syms={'p': 0.01},
        swap_syms={'p': 0.01},
        seed=SEED,
    ),
]
accuracy_top = [1, 5, 10]

## Модуль `api`

In [ ]:
import api

#### Faiss-fasttext

In [ ]:
pipeline1 = api.Pipeline(
    dataset=dataset.target.values,
    preprocessing=preprocessing_list,
    model=model,
    searcher=searcher1,
    verbose=True,
)

In [ ]:
pipeline1.search('mario', 5)

In [ ]:
pipeline1.validate(augmentation_transforms, accuracy_top)

In [ ]:
# pipeline1.save('tmp')

In [ ]:
# pipeline1 = api.load_pipeline('tmp')
# pipeline1

#### KNN-fasttext

In [ ]:
pipeline2 = api.Pipeline(
    dataset=dataset.target.values,
    preprocessing=preprocessing_list,
    model=model,
    searcher=searcher2,
    verbose=True,

    algorithm='brute',
)

In [ ]:
pipeline2.search('mario', 5)

In [ ]:
pipeline2.validate(augmentation_transforms, accuracy_top)